In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/lcp_single_test.xlsx')

In [ ]:
df = df[['corpus', 'sentence']]

In [ ]:
elements = df['corpus'].unique()
print(elements)
label = []
cont = 0
for i in elements:
  label.append(cont)
  cont+=1

print(label)

['bible' 'biomed' 'europarl']
[0, 1, 2]


In [ ]:
for i in range(len(elements)):
  df.loc[df.corpus == elements[int(i)], 'corpus'] = label[int(i)]

In [ ]:
df = df.rename(columns={"corpus": "label", "sentence": "text"})
df

,label,text
0,0,"But he, beckoning to them with his hand to be ..."
1,0,"If I forget you, Jerusalem, let my right hand ..."
2,0,"the ten sons of Haman the son of Hammedatha, t..."
3,0,Let your hand be lifted up above your adversar...
4,0,"Abimelech chased him, and he fled before him, ..."
...,...,...
882,2,"the report by Mr Philippe Busquin, on behalf o..."
883,2,Section V - Court of Auditors (SEC(2002) 405 -...
884,2,"- Mr President, on 1 June, on the basis of inf..."
885,2,I would like to extend a warm welcome to this ...


In [ ]:
!pip install datasets transformers seqeval

     |████████████████████████████████| 306 kB 5.3 MB/s 
     |████████████████████████████████| 3.4 MB 47.2 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.0 MB/s 
     |████████████████████████████████| 243 kB 44.9 MB/s 
     |████████████████████████████████| 61 kB 460 kB/s 
     |████████████████████████████████| 132 kB 46.1 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 32.7 MB/s 
     |████████████████████████████████| 160 kB 38.0 MB/s 
     |████████████████████████████████| 271 kB 48.2 MB/s 
     |████████████████████████████████| 192 kB 45.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=cbce366b317c09d63f7a3cb13da77029741f787895353e0f5f9908338b64f0df
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

In [ ]:
!pip install nlp

     |████████████████████████████████| 1.7 MB 4.7 MB/s 


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
from nlp import Dataset
import pandas as pd
from torch import tensor
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, EvalPrediction
import torch
ds_train = Dataset.from_pandas(df[['text', 'label']])

In [ ]:
type(ds_train)

nlp.arrow_dataset.Dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = ds_train.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tokenized_datasets

Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 887)

In [ ]:
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(400))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(400))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
small_eval_dataset

Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 400)

**Fine-tuning**

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")
#training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 400
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 150


Step,Training Loss


In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
print(metric)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
Examples:

    >>> accuracy_metric = datasets.load_metric("accuracy")
    >>> results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
    >>> print(results)
    {'accuracy': 1.0}
""", stored examples: 0)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 400
  Batch size = 8


{'eval_accuracy': 1.0,
 'eval_loss': 0.0011535807279869914,
 'eval_runtime': 25.9744,
 'eval_samples_per_second': 15.4,
 'eval_steps_per_second': 1.925}